In [ ]:
import pandas as pd
import time

___
# Post-Processing

### (Static) Parameters for post-processing (can be changed to alter design and KPI performance)

In [2]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 200             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
bess_bos= 250               #https://www.energy-storage.news/li-ion-bess-costs-could-fall-47-by-2030-nrel-says-in-long-term-forecast-update/ in EUR/kW
## Also see: https://www.nrel.gov/grid/assets/pdfs/second_grid_sim_zagoras.pdf
# pv_opex= 17                 #EUR/kWh ->reference in excel
# bess_opex= 0.125            #EUR/kWh ->reference in excel
pv_opex= 3
bess_opex= 6

pv_co2=             33      #kgCO2eq/kW_powerDC ->reference in excel
bess_co2=           100     #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2=        0       #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2=      0       #kgCO2eq/kW_powerDC ->assumption
discount_rate=      0.0485  #assumption
degradation_rate=   0.025   #assumption (based on reaching 80% SoH in 8 years)
lifetime_project=   32      #for the project lifetime
lifetime_bess=      8       #for the BESS lifetime

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'bess_bos':         bess_bos,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate
}

### Setting functions for calculating KPIs

In [3]:
## General LCOE function
def calc_lcoe(annual_output, capital_cost, annual_operating_cost, eol_burden, discount_rate, degradation_rate, lifetime):
    discount_factor_annualised = ( (1 - ( 1/( ( 1 + discount_rate ) **lifetime ) ) ) * ( 1 + discount_rate )) / ( ( 1 + discount_rate ) -1 )
    degradation_factor_annualised = ( (1 - ( 1/( ( 1 + degradation_rate ) **lifetime ) ) ) * ( 1 + degradation_rate )) / ( ( 1 + degradation_rate ) -1 )
    result = ( capital_cost + annual_operating_cost * discount_factor_annualised + eol_burden) / ( annual_output * degradation_factor_annualised )
    return result




## Main function for KPIs in the model
### Future update - separate smaller functions
def get_KPIs(
        profile,
        solar_factor,   # solar_factor=   pv_scale
        PessCH,         # PessCH=         p_bess
        EnESS,          # EnESS=          e_bess
        load,
        region
    ):
    '''
    Base columns when profile is loaded:

    'Hour',
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
    'SoC',
    'sum_power_flows'
    '''


    price_curve=    data_dict[region]['Price']/1000
    co2_curve=      data_dict[region]['CO_2_eq']

    t_bess= EnESS/PessCH

    ## Add profits equation, other factors
    savings= 0
    for i in range(8760):
        #savings += ((price_curve.Grid_Price[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
        savings += ((price_curve[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
    
    energy_procurement_costs= 0
    for i in range(8760):
        #energy_procurement_costs += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
        energy_procurement_costs += ((price_curve[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
    
    energy_procurement_costs_load= 0
    for i in range(8760):
        #energy_procurement_costs_load += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_Load[i]))
        energy_procurement_costs_load += ((price_curve[i]) * (profile.P_Grid_to_Load[i]))
    
    energy_procurement_costs_bess= 0
    for i in range(8760):
        #energy_procurement_costs_bess += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_BESS[i]))
        energy_procurement_costs_bess += ((price_curve[i]) * (profile.P_Grid_to_BESS[i]))

    profit= 0
    for i in range(8760):
        #profit += ((price_curve.Grid_Price[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
        profit += ((price_curve[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
    
    profit_pv= 0
    for i in range(8760):
        #profit_pv += ((price_curve.Grid_Price[i]) * (profile.P_PV_to_Grid[i]))
        profit_pv += ((price_curve[i]) * (profile.P_PV_to_Grid[i]))
    
    profit_bess= 0
    for i in range(8760):
        #profit_bess += ((price_curve.Grid_Price[i]) * (profile.P_BESS_to_Grid[i]))
        profit_bess += ((price_curve[i]) * (profile.P_BESS_to_Grid[i]))
    
    co2avoided= 0
    for i in range(8760):
        #co2avoided += ((co2_curve.carbon_intensity_production_avg[i] - 12) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
        #co2avoided += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
        co2avoided += ((co2_curve[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
    
    co2burden= 0
    for i in range(8760):
        #co2burden += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
        co2burden += ((co2_curve[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
    
    co2burden_load= 0
    for i in range(8760):
        #co2burden_load += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_Load[i]))
        co2burden_load += ((co2_curve[i]) * (profile.P_Grid_to_Load[i]))
    
    co2burden_bess= 0
    for i in range(8760):
        #co2burden_bess += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_BESS[i]))
        co2burden_bess += ((co2_curve[i]) * (profile.P_Grid_to_BESS[i]))

    co2abatement= 0
    for i in range(8760):
        #co2abatement += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
        co2abatement += ((co2_curve[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
    
    co2abatement_pv= 0
    for i in range(8760):
        #co2abatement_pv += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_PV_to_Grid[i]))
        co2abatement_pv += ((co2_curve[i]) * (profile.P_PV_to_Grid[i]))
    
    co2abatement_bess= 0
    for i in range(8760):
        #co2abatement_bess += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_BESS_to_Grid[i]))
        co2abatement_bess += ((co2_curve[i]) * (profile.P_BESS_to_Grid[i]))
    
    

    pv2load=    profile['P_PV_to_Load'].sum()
    pv2bess=    profile['P_PV_to_BESS'].sum()
    pv2curtail= profile['P_PV_curtailment'].sum()
    pv2grid=    profile['P_PV_to_Grid'].sum()
    bess2load=  profile['P_BESS_to_Load'].sum()
    bess2grid=  profile['P_BESS_to_Grid'].sum()
    grid2load=  profile['P_Grid_to_Load'].sum()
    grid2bess=  profile['P_Grid_to_BESS'].sum()

    annual_energy= pv2load + bess2load + pv2grid + bess2grid
    annual_energy_enduser= pv2load + bess2load + grid2load

    if EnESS == 0:
        cycles= int(0)
    else:
        cycles= int((int(bess2load) + int(bess2grid)) / (EnESS * 1000))



    ## Re-check capex equations and make corrections (e.g. add BOS, BOP, EoL)
    ## If need be, reference specific equations for LCOE, or parts of LCOE, calculations
    capex= solar_factor * params['pv_price'] * 1000 + EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    capex_pv= solar_factor * params['pv_price'] * 1000
    capex_bess= EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    opex= solar_factor * params['pv_opex'] * 1000 + EnESS * 1000 * params['bess_opex']
    opex_pv= solar_factor * params['pv_opex'] * 1000
    opex_bess= EnESS * 1000 * params['bess_opex']
    opex_investor= opex + energy_procurement_costs_bess
    if EnESS == 0:
        opex_enduser= opex + energy_procurement_costs_load
    elif EnESS == 0.0:
        opex_enduser= opex + energy_procurement_costs_load
    else:
        opex_enduser= opex + energy_procurement_costs_load + energy_procurement_costs_bess * (bess2load/(bess2load + bess2grid))
    eol_capex= capex * 0.3
    capex_co2= solar_factor * params['pv_co2'] * 1000 + EnESS * 1000 * params['bess_co2']
    capex_co2_pv= solar_factor * params['pv_co2'] * 1000
    capex_co2_bess= EnESS * 1000 * params['bess_co2']
    opex_co2= solar_factor * params['pv_opex_co2'] * 1000 + EnESS * 1000 * params['bess_opex_co2']
    opex_co2_pv= solar_factor * params['pv_opex_co2'] * 1000
    opex_co2_bess= EnESS * 1000 * params['bess_opex_co2']
    opex_co2_investor= opex_co2 + co2burden_bess
    if EnESS == 0:
        opex_co2_enduser= opex_co2 + co2burden_load
    elif EnESS == 0.0:
        opex_co2_enduser= opex_co2 + co2burden_load
    else:
        opex_co2_enduser= opex_co2 + co2burden_load + co2burden_bess * (bess2load/(bess2load + bess2grid))
    eol_co2= capex_co2 * 0.3

    ## Get LCOE and LCO2 for the system
    if annual_energy == 0:
        lcoe_revenue=   0
        lcoe_investor=  0
        lco2_abatement= 0
        lco2_investor=  0
    elif annual_energy == 0.0:
        lcoe_revenue=   0
        lcoe_investor=  0
        lco2_abatement= 0
        lco2_investor=  0
    else:
        lcoe_revenue= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex,
            annual_operating_cost= (opex + energy_procurement_costs_bess - profit),
            eol_burden= eol_capex,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lcoe_investor= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex,
            annual_operating_cost= opex_investor,
            eol_burden= eol_capex,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lco2_abatement= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex_co2,
            annual_operating_cost= (opex_co2 + co2burden_bess - co2abatement),
            eol_burden= eol_co2,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lco2_investor= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex_co2,
            annual_operating_cost= opex_co2_investor,
            eol_burden= eol_co2,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )




    lcoe_enduser= calc_lcoe(
        annual_output= annual_energy_enduser,
        capital_cost= capex,
        annual_operating_cost= opex_enduser,
        eol_burden= eol_capex,
        discount_rate= discount_rate,
        degradation_rate= degradation_rate,
        lifetime= 32
    )
    
    lco2_enduser= calc_lcoe(
        annual_output= annual_energy_enduser,
        capital_cost= capex_co2,
        annual_operating_cost= opex_co2_enduser,
        eol_burden= eol_co2,
        discount_rate= discount_rate,
        degradation_rate= degradation_rate,
        lifetime= 32
    )



    ## SS = how much of the load was covered by the PV system
    self_sufficiency= ((pv2load + bess2load) / (loads_selected[load].sum() * 1000)) * 100
    #self_sufficiency= ((pv2load + pv2bess) / data.Load.sum()) * 100
    
    if EnESS == 0:
        ress= (pv2load / (loads_selected[load].sum() * 1000)) * 100
    elif EnESS == 0.0:
        ress= (pv2load / (loads_selected[load].sum() * 1000)) * 100
    elif solar_factor == 0:
        ress = 0
    elif solar_factor == 0.0:
        ress = 0
    else:
        ress= ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / (loads_selected[load].sum() * 1000)) * 100

    ## SC = 1 - curtailment factor?
    if solar_factor == 0:
        self_consumption = 0
    else:
        self_consumption= ((pv2load + pv2bess) / (data_dict[region].solar_pv.sum() * (solar_factor / 100))) * 100
    
    
    
    parameters = (
        solar_factor,
        PessCH,
        EnESS,
        t_bess,
        pv2load,
        pv2bess,
        pv2curtail,
        pv2grid,
        bess2load,
        bess2grid,
        grid2load,
        grid2bess,
        annual_energy,
        capex,
        capex_pv,
        capex_bess,
        opex,
        opex_pv,
        opex_bess,
        capex_co2,
        capex_co2_pv,
        capex_co2_bess,
        opex_co2,
        opex_co2_pv,
        opex_co2_bess,
        lcoe_investor,
        lcoe_enduser,
        lcoe_revenue,
        lco2_investor,
        lco2_enduser,
        lco2_abatement,
        savings,
        energy_procurement_costs,
        energy_procurement_costs_load,
        energy_procurement_costs_bess,
        profit,
        profit_pv,
        profit_bess,
        co2avoided,
        co2burden,
        co2burden_load,
        co2burden_bess,
        co2abatement,
        co2abatement_pv,
        co2abatement_bess,
        cycles,
        self_sufficiency,
        ress,
        self_consumption,
        load,
        region
    )
    return parameters


### Setting DataFrame for results

In [4]:
columns=[
        'P_PV',
        'P_BESS',
        'E_BESS',
        'T_BESS',
        'pv2load',
        'pv2bess',
        'pv2curtail',
        'pv2grid',
        'bess2load',
        'bess2grid',
        'grid2load',
        'grid2bess',
        'annual_energy',
        'capex',
        'capex_pv',
        'capex_bess',
        'opex',
        'opex_pv',
        'opex_bess',
        'capex_co2',
        'capex_co2_pv',
        'capex_co2_bess',
        'opex_co2',
        'opex_co2_pv',
        'opex_co2_bess',
        'lcoe_investor',
        'lcoe_enduser',
        'lcoe_revenue',
        'lco2_investor',
        'lco2_enduser',
        'lco2_abatement',
        'savings',
        'energy_procurement_costs',
        'energy_procurement_costs_load',
        'energy_procurement_costs_bess',
        'profit',
        'profit_pv',
        'profit_bess',
        'co2avoided',
        'co2burden',
        'co2burden_load',
        'co2burden_bess',
        'co2abatement',
        'co2abatement_pv',
        'co2abatement_bess',
        'cycles',
        'self_sufficiency',
        'ress',
        'self_consumption',
        'load_source',
        'region'
    ]

In [5]:
results_econ=            pd.DataFrame(
    columns= columns
)
results_env=             pd.DataFrame(
    columns= columns
)



results_econ_old=            pd.DataFrame(
    columns= columns
)
results_env_old=             pd.DataFrame(
    columns= columns
)

### Specifying ranges of profiles for post-processing

In [6]:
### A bit of an awkward solution for now
## Import resindential and industrial loads
loads_residential= pd.read_csv(
    'Data/LoadProfiles_SE_Residential.csv',
)

loads_to_use_residential= [
    'residential1',
    'residential2',
    'residential3',
    'residential4',
    'residential5'
]



loads_industrial= pd.read_csv(
    'Data/LoadProfiles_SE_Industrial.csv',
)

loads_to_use_industrial= [
    'industrial1',
    'industrial2',
]





## Create a common DataFrame for the loads
loads_selected= pd.merge(
    loads_residential[['Hour', 'residential1', 'residential2', 'residential3', 'residential4','residential5']],
    loads_industrial[['Hour', 'industrial1', 'industrial2']]
)

loads_to_read= [
    'residential1',
    'residential2',
    'residential3',
    'residential4',
    'residential5',
    'industrial1',
    'industrial2',
]





solar_multiples=    [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
bess_multiples=     [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
bess_capacities=    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
optimise_for=       ['Price', 'CO_2_eq']





data_de=    pd.read_csv('Data/data_de.csv', sep= ';', index_col= False)
data_es=    pd.read_csv('Data/data_es.csv', sep= ';', index_col= False)
data_it=    pd.read_csv('Data/data_it.csv', sep= ';', index_col= False)
data_pl=    pd.read_csv('Data/data_pl.csv', sep= ';', index_col= False)
data_se=    pd.read_csv('Data/data_se.csv', sep= ';', index_col= False)
data_se1=   pd.read_csv('Data/data_se1.csv', sep= ';', index_col= False)
data_se2=   pd.read_csv('Data/data_se2.csv', sep= ';', index_col= False)
data_se3=   pd.read_csv('Data/data_se3.csv', sep= ';', index_col= False)
data_se4=   pd.read_csv('Data/data_se4.csv', sep= ';', index_col= False)

data_dict= {
    'DE': data_de,
    'ES': data_es,
    'IT': data_it,
    'PL': data_pl,
    'SE': data_se,
    'SE1': data_se1,
    'SE2': data_se2,
    'SE3': data_se3,
    'SE4': data_se4,
}

regions= [
    'DE',
    'ES',
    'IT',
    'PL',
    'SE',
    'SE1',
    'SE2',
    'SE3',
    'SE4',
]

### Initialising the postprocessor

In [ ]:
start_time= time.time()


for bess_capacity in bess_capacities:
    for solar_multiple in solar_multiples:
        for bess_multiple in bess_multiples:
            for load in loads_to_read:
                for curve in optimise_for:
                    for region in regions:
                        try:
                            solar_factor= solar_multiple# * 3
                            PessCH= bess_multiple# * 3
                            EnESS= PessCH * bess_capacity


                            profile = (pd.read_csv(f'RESULTS/{curve}_{region}_{load}_{solar_multiple}pv_{bess_multiple}bess_{bess_capacity}hr.csv')).fillna(value= 0)
                            profile['P_PV_to_Load']=        profile['P_PV_to_Load'].astype('int')
                            profile['P_PV_to_BESS']=        profile['P_PV_to_BESS'].astype('int')
                            profile['P_PV_curtailment']=    profile['P_PV_curtailment'].astype('int')
                            profile['P_PV_to_Grid']=        profile['P_PV_to_Grid'].astype('int')
                            profile['P_BESS_to_Load']=      profile['P_BESS_to_Load'].astype('int')
                            profile['P_BESS_to_Grid']=      profile['P_BESS_to_Grid'].astype('int')
                            profile['P_Grid_to_Load']=      profile['P_Grid_to_Load'].astype('int')
                            profile['P_Grid_to_BESS']=      profile['P_Grid_to_BESS'].astype('int')
                            #print(profile.info())

                            if curve == 'Price':
                                results_econ.loc[len(results_econ)] = (get_KPIs(profile, solar_factor= solar_factor, PessCH= PessCH, EnESS= EnESS, load= load, region= region))
                            elif curve == 'CO_2_eq':
                                results_env.loc[len(results_env)] = (get_KPIs(profile, solar_factor= solar_factor, PessCH= PessCH, EnESS= EnESS, load= load, region= region))
                        except FileNotFoundError:
                            continue




end_time= time.time()
execution_time= end_time - start_time

# Convert execution time to hours, minutes, and seconds
hours, remainder = divmod(execution_time, 3600)
minutes, seconds = divmod(remainder, 60)



to_read=(len(loads_to_read) * len(solar_multiples) * len(bess_multiples) * len(bess_capacities) * len(regions) * len(optimise_for))
been_read=   (len(results_econ) + len(results_env))
# Print the execution time in the format hh:mm:ss
print('Postprocessing PaperA-selected dispatch profiles')
print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
print(f'Cases to read: {to_read}')
print(f'Cases Read: {been_read}')
print(f'Scenarios Generated: {100 * round((been_read/to_read), 4)}%')

In [10]:
results_econ.to_csv('RESULTS/Results_econ.csv', index= False)
results_env.to_csv('RESULTS/Results_env.csv', index= False)